In [7]:
pip install tensorflow

In [10]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import gensim
from gensim.models import KeyedVectors
from gensim.models import word2vec
import spacy
import gensim.downloader as api
from gensim.models.word2vec import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from google.colab import drive
#!unzip gdrive/My\ Drive/data/train.zip
df = pd.read_csv('/content/drive/MyDrive/cleaned_review.csv')

In [ ]:
df

In [ ]:
df = df.drop(['Unnamed: 0.1'], axis=1)
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df = df.astype({'review': 'str'})

In [9]:
!unzip /content/drive/MyDrive/tweet_cbow_100.zip

Archive:  /content/drive/MyDrive/tweet_cbow_100.zip
  inflating: tweets_cbow_100         
  inflating: tweets_cbow_100.trainables.syn1neg.npy  
  inflating: tweets_cbow_100.wv.vectors.npy  


In [10]:
!unzip /content/drive/MyDrive/fasttext.zip

Archive:  /content/drive/MyDrive/fasttext.zip
  inflating: cc.ar.300.vec           


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
model_wv = gensim.models.Word2Vec.load("tweets_cbow_100")

NameError: name 'gensim' is not defined

**1- Word2Vec**

In [13]:
#adding all reviews into a list of docs
docs = []
for doc in df['review']:
  docs.append(doc)

In [14]:
#tokenize and vectorize the data
def vectorize(text):
    vectors = []
    tokens = word_tokenize(text)
    for token in tokens:
        try:
            vectors.append(model_wv.wv[token])
        except KeyError:
            continue
    return np.mean(vectors, axis=0)


In [15]:
df['vector'] = df['review'].apply(lambda text: vectorize(text))
df

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


,rating,company,review,vector
0,1,talbat,رئع,NaN
1,1,talbat,رنمج رئع جدا سعد لبه حيج شكل سرع,"[1.6961856, 0.08220401, -0.33615932, -0.392516..."
2,-1,talbat,طبق غتح دائ يعط وجد تصل شبك انه الن عند تمم شو...,"[-0.22570415, 0.20668864, 0.05381265, -0.87980..."
3,-1,talbat,لمذ يمك طلب ماكدونالدز؟,"[2.3001964, -4.126458, 0.91109973, 1.2804632, ..."
4,-1,talbat,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر,"[0.89374125, -0.45155722, -0.4417662, -0.02822..."
...,...,...,...,...
38157,1,telecom_egypt,متز عمل متز سعر بلغ شوه,"[1.0109402, -0.6620609, -0.24655917, -0.112242..."
38158,-1,telecom_egypt,رحل كوس حلة باص مش اد كده يري حدث باص حتي تكون...,"[0.7163581, -0.51264083, -0.11336234, -1.27559..."
38159,-1,telecom_egypt,جرب جيد نقص عدم همم خدم عملاء عمل مره وصل شرك ...,"[2.5690944, -0.10699671, 0.34229198, 0.0172564..."
38160,-1,telecom_egypt,جيد لمذ وجد خطط كثر ريط يصل حتي ندي رمايه؟ رجو...,"[0.607037, 0.27123147, 0.27895132, 0.12746403,..."


In [52]:
df = df.dropna()

In [41]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values, 
    df.rating, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df.rating
)

In [64]:
X_train.shape

(27062,)

In [65]:
print("Shape of X_train before reshaping: ", X_train.shape)
print("Shape of X_test before reshaping: ", X_test.shape)


X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

print("Shape of X_train after reshaping: ", X_train_2d.shape)
print("Shape of X_test after reshaping: ", X_test_2d.shape)

Shape of X_train before reshaping:  (27062,)
Shape of X_test before reshaping:  (6766,)
Shape of X_train after reshaping:  (27062, 100)
Shape of X_test after reshaping:  (6766, 100)


Evaluating Word2Vec using Decision Tree

In [56]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

 # Initialize the classifier
clf = DecisionTreeClassifier()

 # Train the classifier
clf.fit(X_train_2d, y_train)

 # Predict the labels of the test set
y_pred = clf.predict(X_test_2d)

# # Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# # Print the accuracy of the model
print("Accuracy:", accuracy)

# # Print the classfication report
print(classification_report(y_test, y_pred))

Accuracy: 0.5161099615725687
              precision    recall  f1-score   support

          -1       0.41      0.40      0.40      2464
           0       0.06      0.06      0.06       329
           1       0.62      0.63      0.62      3973

    accuracy                           0.52      6766
   macro avg       0.36      0.36      0.36      6766
weighted avg       0.51      0.52      0.51      6766



Evaluating using Gradient Boosting

In [66]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.56      0.26      0.36      2464
           0       0.00      0.00      0.00       329
           1       0.63      0.88      0.73      3973

    accuracy                           0.61      6766
   macro avg       0.39      0.38      0.36      6766
weighted avg       0.57      0.61      0.56      6766



**2- Fasttext**

In [20]:
model_ft = KeyedVectors.load_word2vec_format("cc.ar.300.vec")

In [21]:
df2 = df.copy()

In [24]:
df2  = df2.drop(['vector'], axis=1)

In [25]:
docs2 = []
for doc in df2['review']:
  docs2.append(doc)

In [26]:
def vectorize_fasttest(text):
    vectors = []
    tokens = word_tokenize(text)
    for token in tokens:
        try:
            vectors.append(model_ft[token])
        except KeyError:
            continue
    return np.mean(vectors, axis=0)


In [27]:
df2['vector'] = df2['review'].apply(lambda text: vectorize_fasttest(text))
df2

/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


,rating,company,review,vector
0,1,talbat,رئع,"[-0.1145, -0.0614, 0.0576, -0.2185, 0.3668, 0...."
1,1,talbat,رنمج رئع جدا سعد لبه حيج شكل سرع,"[-0.012542859, -0.0003999988, 0.025314286, 0.0..."
2,-1,talbat,طبق غتح دائ يعط وجد تصل شبك انه الن عند تمم شو...,"[-0.038458336, -0.05558333, 0.037875, 0.064475..."
3,-1,talbat,لمذ يمك طلب ماكدونالدز؟,"[0.058966666, -0.0006333335, 0.12616667, 0.026..."
4,-1,talbat,رنمج ظهر طعم غلق انه بتك فتح بقل كده كتر شهر,"[0.01135, -0.040400006, -0.038259998, 0.06482,..."
...,...,...,...,...
38157,1,telecom_egypt,متز عمل متز سعر بلغ شوه,"[-0.017383335, 0.0315, -0.02658333, 0.14483334..."
38158,-1,telecom_egypt,رحل كوس حلة باص مش اد كده يري حدث باص حتي تكون...,"[-0.05492353, -0.024970587, 0.0108, 0.07174118..."
38159,-1,telecom_egypt,جرب جيد نقص عدم همم خدم عملاء عمل مره وصل شرك ...,"[-0.04741177, 0.02565294, -0.009400002, 0.0430..."
38160,-1,telecom_egypt,جيد لمذ وجد خطط كثر ريط يصل حتي ندي رمايه؟ رجو...,"[0.013154546, -0.0057999995, 0.0126363635, 0.0..."


In [59]:
df2 = df2.dropna()

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df2.vector.values, 
    df2.rating, 
    test_size=0.2, # 20% samples will go to test dataset
    random_state=2022,
    stratify=df2.rating
)

In [68]:
print("Shape of X_train before reshaping: ", X_train.shape)
print("Shape of X_test before reshaping: ", X_test.shape)

X_train_2d = np.stack(X_train)
X_test_2d =  np.stack(X_test)

print("Shape of X_train after reshaping: ", X_train_2d.shape)
print("Shape of X_test after reshaping: ", X_test_2d.shape)

Shape of X_train before reshaping:  (29252,)
Shape of X_test before reshaping:  (7313,)
Shape of X_train after reshaping:  (29252, 300)
Shape of X_test after reshaping:  (7313, 300)


Evaluating using Decision Tree

In [69]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

 # Initialize the classifier
clf = DecisionTreeClassifier()

# # Train the classifier
clf.fit(X_train_2d, y_train)

# # Predict the labels of the test set
y_pred = clf.predict(X_test_2d)

# # Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# # Print the accuracy of the model
print("Accuracy:", accuracy)

# # Print the classfication report
print(classification_report(y_test, y_pred))

Accuracy: 0.5044441405715848
              precision    recall  f1-score   support

          -1       0.39      0.39      0.39      2618
           0       0.04      0.05      0.04       353
           1       0.62      0.61      0.61      4342

    accuracy                           0.50      7313
   macro avg       0.35      0.35      0.35      7313
weighted avg       0.51      0.50      0.51      7313



Evaluating using Gradient Boosting

In [70]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

#1. creating a GradientBoosting model object
clf = GradientBoostingClassifier()

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)


#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)


#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.54      0.27      0.36      2618
           0       0.00      0.00      0.00       353
           1       0.63      0.87      0.73      4342

    accuracy                           0.62      7313
   macro avg       0.39      0.38      0.36      7313
weighted avg       0.57      0.62      0.56      7313



In [19]:
# %%shell
# jupyter nbconvert --to html /content/NLP_Task02.ipynb